In [1]:
import numpy as np

def gram_schmidt(A: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Reduced QR factorization of a matrix A using the Gram-Schmidt orthogonalization process.
    """
    Q = np.zeros((A.shape[0], A.shape[1]))
    R = np.zeros((A.shape[1], A.shape[1]))
    for i in range(A.shape[1]):
        Q[:, i] = A[:, i]
        for j in range(i):
            R[j, i] = np.dot(Q[:, j], A[:, i])
            Q[:, i] -= R[j, i] * Q[:, j]
        R[i, i] = np.linalg.norm(Q[:, i])
        Q[:, i] /= R[i, i]
    return Q, R

A = np.array([[3, 7], [0, 12], [4, 1]])
Q, R = gram_schmidt(A)
print(Q)
print(R)

[[ 0.6         0.30769231]
 [ 0.          0.92307692]
 [ 0.8        -0.23076923]]
[[ 5.  5.]
 [ 0. 13.]]


In [5]:
import numpy as np

def householder_qr(A):
    """
    Compute QR factorization using Householder transformation.
    
    Parameters:
    A (numpy.ndarray): Input matrix to factorize
    
    Returns:
    Q (numpy.ndarray): Orthogonal matrix
    R (numpy.ndarray): Upper triangular matrix
    """
    # Get matrix dimensions
    m, n = A.shape
    
    # Create a copy of the input matrix to avoid modifying the original
    R = A.copy().astype(float)
    
    # Initialize Q as the identity matrix
    Q = np.eye(m)
    
    # Iterate through columns to create Householder reflectors
    for k in range(min(m-1, n)):
        # Extract the column we're currently working on
        x = R[k:, k]
        
        # Compute the norm of the column
        norm_x = np.linalg.norm(x)
        
        # Skip if column is already zero
        if norm_x == 0:
            continue
        
        # Compute the Householder vector (reflection vector)
        # Ensure numerical stability by choosing sign that maximizes cancellation
        u = x.copy()
        u[0] -= norm_x * np.sign(x[0])
        
        # Normalize the Householder vector
        v = u / np.linalg.norm(u)
        
        # Reflect the submatrix of R
        # Householder transformation: R = (I - 2vv^T)R
        R[k:, k:] -= 2.0 * np.outer(v, v.T @ R[k:, k:])
        
        # Update Q 
        # Q accumulates all the Householder reflections
        Q_k = np.eye(m)
        Q_k[k:, k:] -= 2.0 * np.outer(v, v)
        Q = Q @ Q_k
    
    # Zero out very small values to improve numerical stability
    R[np.abs(R) < 1e-12] = 0
    
    return Q.T, R

# Verification with increased numerical tolerance
def main():
    # Create a sample matrix
    A = np.array([
        [12, -51, 4], 
        [6, 167, -68], 
        [-4, 24, -41]
    ], dtype=float)
    
    # Compute QR factorization
    Q, R = householder_qr(A)
    
    # Verify the factorization with increased tolerance
    print("Original Matrix A:")
    print(A)
    print("\nOrthogonal Matrix Q:")
    print(Q)
    print("\nUpper Triangular Matrix R:")
    print(R)
    
    # Check properties: A = QR with a more lenient tolerance
    print("\nVerification: A ≈ QR")
    print(np.allclose(A, Q @ R, atol=1e-10, rtol=1e-10))
    
    # Check orthogonality of Q: Q^T * Q = I
    print("\nOrthogonality Check (Q^T * Q ≈ I):")
    print(np.allclose(Q.T @ Q, np.eye(Q.shape[0])))

if __name__ == "__main__":b
    main()

Original Matrix A:
[[ 12. -51.   4.]
 [  6. 167. -68.]
 [ -4.  24. -41.]]

Orthogonal Matrix Q:
[[ 0.85714286  0.42857143 -0.28571429]
 [ 0.39428571 -0.90285714 -0.17142857]
 [-0.33142857  0.03428571 -0.94285714]]

Upper Triangular Matrix R:
[[  14.   21.  -14.]
 [   0. -175.   70.]
 [   0.    0.   35.]]

Verification: A ≈ QR
False

Orthogonality Check (Q^T * Q ≈ I):
True


In [1]:
# Experimentally verify backward stability of the Householder QR factorization
# Generate matrix A with known QR factorization

import numpy as np
R = np.triu(np.random.rand(50,50)) 
Q = np.linalg.qr(np.random.rand(50,50))[0] 
A = Q@R

Q2, R2 = np.linalg.qr(A)

# Compute forward and backward errors
forward_error_Q = np.linalg.norm(Q - Q2) / np.linalg.norm(Q)
forward_error_R = np.linalg.norm(R - R2) / np.linalg.norm(R)
backward_error = np.linalg.norm(A - Q2@R2) / np.linalg.norm(A)

print(f"Forward error in Q: {forward_error_Q:.2e}")
print(f"Forward error in R: {forward_error_R:.2e}")
print(f"Backward error: {backward_error:.2e}")

# Compute condition number of A
cond_A = np.linalg.cond(A)
print(f"\nCondition number of A: {cond_A:.2e}")

# Interpretation
print("\nInterpretation:")
print("1. The backward error is very small, showing the computed QR")
print("   factorization is backward stable (QR ≈ A)")
print("2. The forward errors in Q and R are larger than the backward")
print("   error but still reasonable, indicating some sensitivity")
print("3. The condition number gives us the worst-case amplification")
print("   of relative errors, explaining the relationship between")
print("   forward and backward errors")




Forward error in Q: 8.28e-10
Forward error in R: 2.12e-10
Backward error: 5.62e-16

Condition number of A: 5.68e+09

Interpretation:
1. The backward error is very small, showing the computed QR
   factorization is backward stable (QR ≈ A)
2. The forward errors in Q and R are larger than the backward
   error but still reasonable, indicating some sensitivity
3. The condition number gives us the worst-case amplification
   of relative errors, explaining the relationship between
   forward and backward errors


In [5]:
# Define a full rank matrix A in C
import numpy as np
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
b = np.array([1, 2, 3])
def solve_qr_householder(A, b):
    Q,R = np.linalg.qr(A)
    # if m > n, compute least squares solution
    if A.shape[0] == A.shape[1]:
        # solve Ax = b using QR factorization
        x_with_A = np.linalg.solve(A, b)
        # instead of Ax = b we use QR factorization so we solve Rx = Q^T @ b
        x_with_Q = np.linalg.solve(R, Q.T @ b)
    else:
        # solve least squares problem
        x_with_A = np.linalg.lstsq(A, b, rcond=None)[0]
        # instead of Ax = b we use QR factorization so we solve Rx = Q^T @ b
        x_with_Q = np.linalg.lstsq(R, Q.T @ b, rcond=None)[0]

    # print both solutions and the relative and absolute errors
    print(f"x_with_A: {x_with_A}")
    print(f"x_with_Q: {x_with_Q}")
    print(f"Relative error: {np.linalg.norm(x_with_A - x_with_Q) / np.linalg.norm(x_with_A)}")
    print(f"Absolute error: {np.linalg.norm(x_with_A - x_with_Q)}")
    return x_with_A, x_with_Q
    
x_with_A, x_with_Q = solve_qr_householder(A, b)




x_with_A: [-3.33333333e-01  6.66666667e-01  3.17206578e-17]
x_with_Q: [-3.33333333e-01  6.66666667e-01 -1.22376596e-15]
Relative error: 2.6816760400451285e-15
Absolute error: 1.9988033063911186e-15
